In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# 기본 정보 크롤링

In [3]:
month_list = []
week_list = []
rank_list = []
id_list = []
title_list = []
author_list = []
publisher_list = []
price_list = []

In [4]:
for i in range(1,11):
  for j in range(1,6):
    for m in range(1,6):
      url = f'http://www.yes24.com/24/category/bestseller?CategoryNumber=001001046011&sumgb=08&year=2022&month={i}&week={j}&day=1&PageNumber={m}&layout=3'
      page = requests.get(url)
      soup = BeautifulSoup(page.content, 'html.parser')
      for k in range(1,21):
        if soup.select('#category_layout') == []:
          pass
        else:
          title = soup.select(f'#category_layout > tr:nth-of-type({k}) > td:nth-of-type(2) > a:nth-of-type(1)')
          author = soup.select(f'#category_layout > tr:nth-of-type({k}) > td:nth-of-type(2) > p > a:nth-of-type(1)')
          price = soup.select(f'#category_layout > tr:nth-of-type({k}) > td:nth-of-type(2) > span > strong')

          id = re.sub(r'[^0-9]','',str(title).split('>')[0])
          
          month_list.append(f'{i}')
          week_list.append(f'{j}')
          rank_list.append(f'{k+20*(m-1)}')
          id_list.append(id)
          title_list.append(str(title).split('>')[1].split('<')[0].strip())
          author_list.append(str(author).split('>')[1].split('<')[0].strip())
          price_list.append(str(price).split('>')[1].split('<')[0].strip())

In [5]:
# 1차 df 정리

In [6]:
df = pd.DataFrame(zip(month_list,week_list,rank_list,id_list,title_list,author_list,price_list))
df.columns = ['month','week','rank','product_id','title','author','price']
df = df[['product_id','month','week','rank','title','author','price']]

In [7]:
# 책 구체 장르 크롤링

In [8]:
genres = ['추리/미스터리','공포/스릴러','판타지','무협','SF','역사','로맨스']
genre_list = []
id_set = list(set(id_list))

In [9]:
for i in id_set:
  url = f'http://www.yes24.com/Product/Goods/{i}'
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  genre = soup.select('#infoset_goodsCate > div.infoSetCont_wrap > dl > dd > ul')
  for j in genres:
    if j in str(genre):
      genre_list.append(f'{i}_{j}')
    elif genre == []:
      genre_list.append(f'{i}_x')

In [10]:
# x 발생 원인은 성인 제한으로 페이지가 정상적으로 넘어가지지 않아서 발생
# 따로 로그인 후 장르를 직접 입력

In [11]:
genre_id = []
genre_first = []

In [12]:
for i in genre_list:
  genre_id.append(i.split('_')[0])
  genre_first.append(i.split('_')[1])

In [13]:
df2 = pd.DataFrame(zip(genre_id,genre_first))
df2.drop_duplicates(inplace=True)
df2 = df2.sort_values(by=0).reset_index(drop=True)

In [14]:
df3 = df2.copy()
df3.drop_duplicates(subset=0, inplace=True) # 첫번째 장르

In [15]:
df4 = df2.copy()
df4 = df4.drop(df3.index)
df4.drop_duplicates(subset=0, inplace=True) # 두번째 장르 가진 애들

In [16]:
df5 = df2.copy()
df5.drop(df3.index, inplace=True)
df5.drop(df4.index, inplace=True) # 세번째 장르 가진 애들

In [17]:
df3[df3.loc[:,1] == 'x'].index

Int64Index([127, 133, 134, 189, 190, 290], dtype='int64')

In [18]:
# 127 108708593 로맨스
# 133 108934285 로맨스
# 134 108934329 로맨스
# 189 110529297 로맨스
# 190 110556884 추리/미스터리, 공포/스릴러
# 290 28473670 공포/스릴러

In [19]:
# 첫번째 장르 변경

In [20]:
def genre_change(id, genre):
  df3.loc[id,1] = genre

In [21]:
genre_change(127, '로맨스')
genre_change(133, '로맨스')
genre_change(134, '로맨스')
genre_change(189, '로맨스')
genre_change(190, '추리/미스터리')
genre_change(290, '공포/스릴러')

In [22]:
# 두번째 장르 변경

In [23]:
df4.loc[438] = [110556884, '공포/스릴러']

In [24]:
# 2차 df 정리

In [25]:
df3 = df3.reset_index(drop=True)
df4 = df4.reset_index(drop=True)
df5 = df5.reset_index(drop=True)

In [61]:
df3.columns = ['product_id','genre']
df4.columns = ['product_id','genre_second']
df5.columns = ['product_id','genre_third']

In [27]:
df['price'] = df['price'].str.replace(",", "")
df['price'] = df['price'].str.replace("원", "")

In [28]:
df['product_id'] = df['product_id'].astype(int)
df['month'] = df['month'].astype(int)
df['week'] = df['week'].astype(int)
df['rank'] = df['rank'].astype(int)
df['price'] = df['price'].astype(int)

In [29]:
df3['product_id'] = df3['product_id'].astype(int)
df4['product_id'] = df4['product_id'].astype(int)
df5['product_id'] = df5['product_id'].astype(int)

In [36]:
df6 = df[['product_id','title','author','price']]

In [37]:
df7 = df[['product_id','month','week','rank']]

In [43]:
df6 = df6.drop_duplicates()

In [52]:
df6 = df6.sort_values(by='product_id').reset_index(drop=True)

In [53]:
df3 = df3.sort_values(by='product_id').reset_index(drop=True)

In [62]:
df6['genre_first'] = df3['genre']

In [65]:
df6 = df6.drop('genre', axis=1)

In [66]:
df6.to_csv('./book.csv', index=False)
df7.to_csv('./weekly_rank.csv', index=False)
df4.to_csv('./genre_second.csv', index=False)
df5.to_csv('./genre_third.csv', index=False)